In [15]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import datetime as dt

In [16]:
data = pd.read_csv("F:\\JOb Projects\\Python\\NSE_RELIANCE_EQ.csv")
#print(data.head())

In [17]:
data.drop(columns=['Volume',"Symbol"], inplace=True)


<ul>
<li>Check Color of the candle
<li>Check First 10 min High and Low
<li>Add moving average
<li>Get Long Short Signal 
</ul>

In [18]:
data['Datetime'] = pd.to_datetime(data['Datetime'])

data['Color_10_min'] = np.where(
    (data['Open'].shift(1)>data['Close'])&
     (data['Datetime'].dt.minute == 20)&
     (data['Datetime'].dt.hour == 9) ,"r",
     np.where((data['Open'].shift(1)<data['Close'])&
     (data['Datetime'].dt.minute == 20)&
     (data['Datetime'].dt.hour == 9),'g',np.nan))



Check First 10 min High and Low

In [19]:
data['Prv_high'] = data['High'].shift(1)
data['Prv_low'] = data['Low'].shift(1)
data['10_min_high'] = np.where(
    (data['Datetime'].dt.minute==20)&
    (data['Datetime'].dt.hour==9)&
    (data['Color_10_min'] != np.nan)&
    (data['High'].shift(1)>data['High']), 
    data['High'].shift(1),
    np.where(
        (data['Datetime'].dt.minute==20)&
        (data['Datetime'].dt.hour==9)&
        (data['Color_10_min'] != np.nan)&
        (data['High'].shift(1)<data['High']), data['High'],np.nan)
    )
data['10_min_low'] = np.where(
    (data['Datetime'].dt.minute==20)&
    (data['Datetime'].dt.hour==9)&
    (data['Color_10_min'] != np.nan)&
    (data['Low'].shift(1)>data['Low']), 
    data['Low'],
    np.where(
        (data['Datetime'].dt.minute==20)&
        (data['Datetime'].dt.hour==9)&
        (data['Color_10_min'] != np.nan)&
        (data['Low'].shift(1)<data['Low']), data['Low'].shift(1),np.nan)
    )

#print(data[['Open',"High","Low","Close","10_min_high","10_min_low"]])


Add SMA 


In [20]:
data['SMA_20'] = ta.sma(data['Close'], 20)
data['SMA_50'] = ta.sma(data['Close'], 50)
data = data[data['SMA_50']>=0]
"""for a in list(data.columns):
    try:
        data[a]=round(data[a],1)
    except:
        None"""



'for a in list(data.columns):\n    try:\n        data[a]=round(data[a],1)\n    except:\n        None'

Geting Signals

In [21]:
data['Long/Short'] = np.where(
    (data['Close']<data['SMA_20'])&
    (data['Close']<data['SMA_50'])&
    (data['Open']>data['SMA_20'])&
    (data['Color_10_min']=="r"),
    -1,
    np.where(
        (data['Close']>data['SMA_20'])&
    (data['Close']>data['SMA_50'])&
    (data['Open']>data['SMA_20'])&
    (data['Color_10_min']=="g"),
    1,np.nan
    )
)


In [22]:
#data[data['Datetime'].dt.date==dt.date(2024,3,20)][['Datetime',"Open","High","Low","Close","Long/Short",'Color_10_min']]

In [23]:
color = None

high = None
low = None
date_ = None
postition= None
signal_generated = False
long_short = None
trade_history = pd.DataFrame(columns=["Datetime", "Entry/Exit","Position_Type","Entry_Price","Take_Profit","Take_SL","qty","EXIT_PRICE",'pnl'])
entry_price = 0
sl = 0
tp = 0
initial_capital = 10000



In [24]:
for a in data.iterrows():
    tick = a[1]
    datetime_ = dt.datetime.strptime(str(tick.iloc[0]),'%Y-%m-%d %H:%M:%S')
    time = datetime_.time()
    if datetime_.date() !=date_:
        date_ = datetime_.date()
        position  = None
        long_short = None
        signal_generated = False
    
    if time.hour==9 and time.minute==20:
        if tick.iloc[5] == "g" or tick.iloc[5]=='r':
            
            color=tick.iloc[5]
            high = tick.iloc[2]
            low = tick.iloc[3]
            signal_generated = True
            long_short = tick.iloc[-1]
            #print(date_, color,high,low, long_short)
   
    if signal_generated:
        
        if postition == None:
            if  dt.time(9,20) < datetime_.time() < dt.time(12,30):

                
                if long_short == 1 and tick.iloc[4] > high:
                    entry_price = tick.iloc[4]
                    rtp = 0.01*initial_capital
                    
                    sl = tick.iloc[3]
                    tp = (entry_price-sl)*3+entry_price
                    qty = int(rtp/(entry_price-sl))
                    initial_capital -= qty*entry_price
                    trade_history.loc[len(trade_history)] = [
                        datetime_,'Entry',"Long",
                        entry_price,tp,sl,qty,0
                                                                ]
                    postition=True
                    #print(trade_history)
        elif postition==True:
            #print(datetime_, entry_price, tp, sl, tick.iloc[4])
            if tick.iloc[4]>=tp:
                trade_history.loc[len(trade_history)-1,"EXIT_PRICE"] = tick.iloc[4]
                trade_history.loc[len(trade_history)-1,"pnl"] = entry_price-tick.iloc[4]
                initial_capital+=qty*tick.iloc[4]
                #print(trade_history)
                """trade_history.loc[len(trade_history)] = [
                    datetime_,'Exit',None,
                    None,None,None,tick.iloc[4]

                ]"""
                #print(trade_history)
                postition=None
                high=None
                low=None
                signal_generated=False
                
            elif tick.iloc[4]<=sl:
                trade_history.loc[len(trade_history)-1,"EXIT_PRICE"] = tick.iloc[4]
                trade_history.loc[len(trade_history)-1,"pnl"] = entry_price-tick.iloc[4]
                initial_capital+=qty*tick.iloc[4]
                #print(trade_history)
                postition=None
                high=None
                low=None
                signal_generated=False
            elif datetime_.time()>=dt.time(15,0):
                trade_history.loc[len(trade_history)-1,"EXIT_PRICE"] = tick.iloc[4]
                trade_history.loc[len(trade_history)-1,"pnl"] = entry_price-tick.iloc[4]
                initial_capital+=qty*tick.iloc[4]
                #print(trade_history)
                postition=None
                high=None
                low=None
                signal_generated=False
                
                

            
    #2017-07-04
    
    

C:\Users\Admin\AppData\Local\Temp\ipykernel_14720\1628783198.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '700.26' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  trade_history.loc[len(trade_history)-1,"EXIT_PRICE"] = tick.iloc[4]


In [25]:
trade_history

,Datetime,Entry/Exit,Position_Type,Entry_Price,Take_Profit,Take_SL,qty,EXIT_PRICE
0,2017-07-04 09:30:00,Entry,Long,694.17,700.11,692.19,50,700.26
1,2017-07-05 09:40:00,Entry,Long,713.96,717.32,712.84,92,712.35
2,2017-07-12 09:25:00,Entry,Long,753.61,763.66,750.26,30,750.07
3,2017-07-14 11:10:00,Entry,Long,756.98,763.37,754.85,47,759.06
4,2017-07-17 10:10:00,Entry,Long,767.18,771.86,765.62,65,769.28
...,...,...,...,...,...,...,...,...
370,2024-02-21 09:25:00,Entry,Long,2950.90,2956.75,2948.95,114,2945.40
371,2024-02-29 10:05:00,Entry,Long,2953.00,2977.60,2944.80,26,2940.75
372,2024-03-01 09:35:00,Entry,Long,2934.95,2945.45,2931.45,61,2948.15
373,2024-03-18 10:15:00,Entry,Long,2861.70,2896.50,2850.10,19,2848.55


In [26]:
trade_history['_wins'] = np.where(trade_history['Entry_Price']<trade_history['EXIT_PRICE'],
                     1,0)
print(f"Win:{trade_history['_wins'].sum()}\nLose:{len(trade_history)-trade_history['_wins'].sum()}")
print(f"Accuracy:{trade_history['_wins'].sum()/(len(trade_history)-trade_history['_wins'].sum())}")

Win:142
Lose:233
Accuracy:0.6094420600858369


In [27]:
"""initial_capital = 10000
curve = [initial_capital]
qty = []
for i in trade_history.iterrows():
    print(i[1])
    qty.append(curve[-1]/(i[1].iloc[3] - i[1].iloc[5]))
    initial_capital-=qty[-1]*i[1].iloc[3]
    initial_capital+=qty[-1]*(i[1].iloc[3]-i[1].iloc[4])
    curve.append(initial_capital)
    """

trade_history.to_excel("trade_history.xlsx", index=False)
print(trade_history)


               Datetime Entry/Exit Position_Type  Entry_Price  Take_Profit  \
0   2017-07-04 09:30:00      Entry          Long       694.17       700.11   
1   2017-07-05 09:40:00      Entry          Long       713.96       717.32   
2   2017-07-12 09:25:00      Entry          Long       753.61       763.66   
3   2017-07-14 11:10:00      Entry          Long       756.98       763.37   
4   2017-07-17 10:10:00      Entry          Long       767.18       771.86   
..                  ...        ...           ...          ...          ...   
370 2024-02-21 09:25:00      Entry          Long      2950.90      2956.75   
371 2024-02-29 10:05:00      Entry          Long      2953.00      2977.60   
372 2024-03-01 09:35:00      Entry          Long      2934.95      2945.45   
373 2024-03-18 10:15:00      Entry          Long      2861.70      2896.50   
374 2024-03-20 09:30:00      Entry          Long      2860.60      2878.60   

     Take_SL  qty  EXIT_PRICE  _wins  
0     692.19   50      7